# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [29]:
# import libraries
import pandas as pd
import sqlite3
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from gensim.models import Word2Vec

In [2]:
# load data from database
conn = sqlite3.connect('../data/messages_categories.db')
sqlquery = """SELECT * FROM clean_data;"""
df = pd.read_sql(sqlquery, con=conn)
# Since the local machine can't handle large dataframes, we'll reduce the size of it:
df = df.sample(frac=0.1, random_state=42)
print(df.shape)
x_cols = ["message"]
y_cols = [col for col in df.columns if col not in ["id","message", "original", "genre"]]

X = df[x_cols].squeeze()
y = df[y_cols]

(2622, 40)


In [3]:
X.head()

7917             Please ,give me some help.Thanks before. 
20913    People are sleeping in rudimentary shelters es...
22523    09/03/2013 04:09 GMT by Shingo ITO TOKYO, Sept...
18442    The 15-nation Economic Community of West Afric...
1336     and in addition what time will it be possible ...
Name: message, dtype: object

In [4]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
7917,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20913,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22523,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18442,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        if tok in stopwords.words("english"):
            continue
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ("counts", CountVectorizer(tokenizer=tokenize)),
        ("tfidf", TfidfTransformer()),
        ("classifier", MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(2097,)
(525,)
(2097, 36)


In [8]:
pipeline.fit(X_train, y_train)

/Users/heribertroig/opt/anaconda3/envs/udacity-courses/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('counts',
                 CountVectorizer(tokenizer=<function tokenize at 0x7ff6191c95a0>)),
                ('tfidf', TfidfTransformer()),
                ('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_predict = pipeline.predict(X_test)
y_predict = pd.DataFrame(y_predict, columns=y_test.columns)

In [10]:
def display_results(y_test, y_pred):
    accuracies = {}
    sizes = {}
    total_accuracy = 0
    total_size = 0
    for label in y_test.columns:
        true_labels = np.array(y_test[label])
        predicted_labels = np.array(y_pred[label])
        size = len(true_labels)
        sizes[label] = size

        conf_matrix = confusion_matrix(true_labels, predicted_labels)
        print(f"Confusion Matrix for Label {label}:\n{conf_matrix}\n")
        accuracy = (true_labels == predicted_labels).mean()
        print("Accuracy:", accuracy)
        accuracies[label] = accuracy
        total_accuracy += accuracy * size
        total_size += size
        mean_weighted_accuracy = total_accuracy / total_size
        print(f"\n Mean weighted accuracy: {mean_weighted_accuracy}\n")

    return mean_weighted_accuracy, accuracies, sizes


In [11]:
mean_weighted_accuracy, _, _ = display_results(y_test, y_predict)

Confusion Matrix for Label related:
[[ 26 108   0]
 [ 11 375   0]
 [  0   5   0]]

Accuracy: 0.7638095238095238

 Mean weighted accuracy: 0.7638095238095238

Confusion Matrix for Label request:
[[439   5]
 [ 48  33]]

Accuracy: 0.8990476190476191

 Mean weighted accuracy: 0.8314285714285714

Confusion Matrix for Label offer:
[[522   0]
 [  3   0]]

Accuracy: 0.9942857142857143

 Mean weighted accuracy: 0.8857142857142857

Confusion Matrix for Label aid_related:
[[268  41]
 [ 94 122]]

Accuracy: 0.7428571428571429

 Mean weighted accuracy: 0.85

Confusion Matrix for Label medical_help:
[[476   3]
 [ 45   1]]

Accuracy: 0.9085714285714286

 Mean weighted accuracy: 0.8617142857142858

Confusion Matrix for Label medical_products:
[[503   0]
 [ 22   0]]

Accuracy: 0.9580952380952381

 Mean weighted accuracy: 0.8777777777777778

Confusion Matrix for Label search_and_rescue:
[[511   1]
 [ 13   0]]

Accuracy: 0.9733333333333334

 Mean weighted accuracy: 0.8914285714285715

Confusion Matrix for

### 6. Improve your model
Use grid search to find better parameters.

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('counts',
   CountVectorizer(tokenizer=<function tokenize at 0x7ff6191c95a0>)),
  ('tfidf', TfidfTransformer()),
  ('classifier', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'counts': CountVectorizer(tokenizer=<function tokenize at 0x7ff6191c95a0>),
 'tfidf': TfidfTransformer(),
 'classifier': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'counts__analyzer': 'word',
 'counts__binary': False,
 'counts__decode_error': 'strict',
 'counts__dtype': numpy.int64,
 'counts__encoding': 'utf-8',
 'counts__input': 'content',
 'counts__lowercase': True,
 'counts__max_df': 1.0,
 'counts__max_features': None,
 'counts__min_df': 1,
 'counts__ngram_range': (1, 1),
 'counts__preprocessor': None,
 'counts__stop_words': None,
 'counts__strip_accents': None,
 'counts__token_pattern': '(?u)\\b\\w\\w+\\b',
 'counts__tokenizer': <function __main__.tokenize(text)>,
 'counts__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth

In [13]:
def build_model(parameters):
    pipeline = Pipeline([
        ("counts", CountVectorizer(tokenizer=tokenize)),
        ("tfidf", TfidfTransformer()),
        ("classifier", MultiOutputClassifier(RandomForestClassifier()))
    ])

    # create grid search object
    cv = GridSearchCV(pipeline, parameters)

    return cv

In [14]:
parameters = {'classifier__estimator__min_samples_leaf': [1, 2],
              'classifier__estimator__min_samples_split': [2, 4, 8],
              'classifier__estimator__n_estimators': [50, 100, 150]}



In [15]:
# model = build_model()
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
#
# display_results(model, y_test, y_pred)

In [62]:
x_cols = ["message", "genre"]
X = df[x_cols]
y = df[y_cols]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
7917,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20913,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22523,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18442,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
preprocessor = ColumnTransformer(
    transformers=[
        ('text', Pipeline([
            ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
            ('tfidf_transformer', TfidfTransformer())
        ]), 'message'),
        ('genre', OneHotEncoder(), ['genre'])
        # ('category', 'passthrough', ['genre'])  # Include the new categorical column
    ])
classifier = MultiOutputClassifier(RandomForestClassifier())
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", classifier)
])
pipeline.fit(X_train, y_train)


/Users/heribertroig/opt/anaconda3/envs/udacity-courses/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('text',
                                                  Pipeline(steps=[('count_vectorizer',
                                                                   CountVectorizer(tokenizer=<function tokenize at 0x7ff6191c95a0>)),
                                                                  ('tfidf_transformer',
                                                                   TfidfTransformer())]),
                                                  'message'),
                                                 ('genre', OneHotEncoder(),
                                                  ['genre'])])),
                ('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [26]:
y_predict = pipeline.predict(X_test)
y_predict = pd.DataFrame(y_predict, columns=y_test.columns)

In [27]:
display_results(y_test, y_predict)

Confusion Matrix for Label related:
[[ 30 104   0]
 [  6 380   0]
 [  0   5   0]]

Accuracy: 0.780952380952381

 Mean weighted accuracy: 0.780952380952381

Confusion Matrix for Label request:
[[436   8]
 [ 50  31]]

Accuracy: 0.8895238095238095

 Mean weighted accuracy: 0.8352380952380952

Confusion Matrix for Label offer:
[[522   0]
 [  3   0]]

Accuracy: 0.9942857142857143

 Mean weighted accuracy: 0.8882539682539683

Confusion Matrix for Label aid_related:
[[262  47]
 [ 95 121]]

Accuracy: 0.7295238095238096

 Mean weighted accuracy: 0.8485714285714285

Confusion Matrix for Label medical_help:
[[476   3]
 [ 45   1]]

Accuracy: 0.9085714285714286

 Mean weighted accuracy: 0.8605714285714285

Confusion Matrix for Label medical_products:
[[503   0]
 [ 22   0]]

Accuracy: 0.9580952380952381

 Mean weighted accuracy: 0.8768253968253968

Confusion Matrix for Label search_and_rescue:
[[511   1]
 [ 13   0]]

Accuracy: 0.9733333333333334

 Mean weighted accuracy: 0.8906122448979592

Confusio

(0.9418518518518518,
 {'related': 0.780952380952381,
  'request': 0.8895238095238095,
  'offer': 0.9942857142857143,
  'aid_related': 0.7295238095238096,
  'medical_help': 0.9085714285714286,
  'medical_products': 0.9580952380952381,
  'search_and_rescue': 0.9733333333333334,
  'security': 0.9847619047619047,
  'military': 0.9714285714285714,
  'child_alone': 1.0,
  'water': 0.940952380952381,
  'food': 0.9352380952380952,
  'shelter': 0.9371428571428572,
  'clothing': 0.9809523809523809,
  'money': 0.9847619047619047,
  'missing_people': 0.9866666666666667,
  'refugees': 0.9657142857142857,
  'death': 0.9504761904761905,
  'other_aid': 0.8609523809523809,
  'infrastructure_related': 0.9466666666666667,
  'transport': 0.9485714285714286,
  'buildings': 0.96,
  'electricity': 0.9676190476190476,
  'tools': 0.9942857142857143,
  'hospitals': 0.9961904761904762,
  'shops': 0.9980952380952381,
  'aid_centers': 0.9942857142857143,
  'other_infrastructure': 0.9542857142857143,
  'weather_rel

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [43]:
class Word2VecVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, size=100, window=5, min_count=1, workers=4):
        self.size = size
        self.window = window
        self.min_count = min_count
        self.workers = workers
        self.model = None

    def fit(self, X, y=None):
        messages = X.apply(lambda row: tokenize(row["message"]), axis=1)
        self.model = Word2Vec(sentences=messages, vector_size=self.size, window=self.window,
                              min_count=self.min_count, workers=self.workers)
        return self

    def transform(self, X):
        vectors = []
        for _, row in X.iterrows():
            message = word_tokenize(row['message'])
            message_vectors = np.array([self.model.wv[word] for word in message if word in self.model.wv])
            if message_vectors.size == 0:
                message_vectors = np.zeros((1, self.size))  # Use zero vectors for messages with no known words
            avg_vector = np.mean(message_vectors, axis=0)
            vectors.append(avg_vector)

        return np.vstack(vectors)

In [44]:
preprocessor = ColumnTransformer(
    transformers=[
        ('message', Word2VecVectorizer(), 'message'),
        ('genre', OneHotEncoder(), ['genre'])
    ]
)

# Combine preprocessor with the classifier
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ("classifier", MultiOutputClassifier(RandomForestClassifier()))
])

In [45]:
pipeline.fit(X_train, y_train)

TypeError: Word2VecVectorizer.fit.<locals>.<lambda>() got an unexpected keyword argument 'axis'

In [ ]:
y_predict = pipeline.predict(X_test)
y_predict = pd.DataFrame(y_predict, columns=y_test.columns)
display_results(y_test, y_predict)

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.